In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import visual_behavior.data_access.loading as loading
import visual_behavior.visualization.qc.experiment_plots as ep
import visual_behavior.visualization.qc.container_plots as cp

import visual_behavior.visualization.ophys.summary_figures as sf
import visual_behavior.visualization.utils as utils

Note: there is a bug in the SDK that has not yet been fixed, so you need to use a branch of Marina's fork here: 
    https://github.com/matchings/AllenSDK/tree/rc/2.13.3

In [4]:
import allensdk
allensdk.__version__

'2.13.3'

### get data cache using `from_lims` method

In [5]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

cache = VisualBehaviorOphysProjectCache.from_lims()

### get experiments table for learning mFISH pilot experiments

`ophys_experiment_table` contains a record of each unique 2P field of view recorded in each session. 
One `ophys_session_id` can have 8 imaging planes, corresonding to 8 `ophys_experiment_ids`. 
A given imaging plane that is recorded across multiple sessions is linked by its `ophys_container_id`. 

In [6]:
experiments_table = cache.get_ophys_experiment_table(passed_only=False)

experiments = experiments_table[experiments_table.project_code=='LearningmFISHTask1A']
experiments.head()

,equipment_name,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,cre_line,...,session_name,isi_experiment_id,imaging_depth,targeted_structure,published_at,date_of_acquisition,session_type,experience_level,passive,image_set
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
1154288458,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,165,VISp,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288461,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,265,VISp,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288463,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,170,VISl,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288465,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,270,VISl,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288467,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,175,VISal,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i


### get behavior sessions for our special mouse

In [19]:
behavior_sessions_table = cache.get_behavior_session_table(passed_only=False)

In [20]:
mouse_beh_data = behavior_sessions_table[behavior_sessions_table.mouse_id=='603892'].sort_values(by='date_of_acquisition')

In [21]:
# lets see the training history for this mouse 
mouse_beh_data[['mouse_id', 'session_type', 'ophys_experiment_id']]

,mouse_id,session_type,ophys_experiment_id
behavior_session_id,,,
1153004187,603892,TRAINING_0_gratings_autorewards_15min,NaN
1153543065,603892,TRAINING_1_gratings,NaN
1153793704,603892,TRAINING_1_gratings,"[1153920566, 1153920568, 1153920569, 115392057..."
1154034257,603892,TRAINING_2_gratings_flashed,"[1154288458, 1154288461, 1154288463, 115428846..."
1154262140,603892,TRAINING_2_gratings_flashed,"[1154369474, 1154369476, 1154369477, 115436947..."
1154472358,603892,TRAINING_3_images_A_10uL_reward,"[1154572286, 1154572288, 1154572289, 115457229..."
1155069835,603892,TRAINING_3_images_A_10uL_reward,"[1155282289, 1155282293, 1155282294, 115528229..."
1155426295,603892,TRAINING_4_images_A_training,"[1155524637, 1155524639, 1155524640, 115552464..."
1155634565,603892,TRAINING_5_images_A_epilogue,"[1155760201, 1155760204, 1155760205, 115576020..."


In [9]:
behavior_session_ids = mouse_beh_data.index.values

### load one behavior session and get relevant data

In [10]:
behavior_session_id = behavior_session_ids[5]

behavior_session = cache.get_behavior_session(behavior_session_id)

In [11]:
behavior_session.running_speed

,timestamps,speed
0,0.021150,-0.025884
1,0.037830,6.817333
2,0.054512,13.332567
3,0.071193,19.200593
4,0.087873,24.152843
...,...,...
218254,3640.690396,-0.064039
218255,3640.707078,-0.057213
218256,3640.723760,-0.043798
218257,3640.740481,-0.025100


In [15]:
behavior_session.licks

,timestamps,frame
0,8.061200,482
1,8.228002,492
2,20.621691,1235
3,23.640903,1416
4,29.512453,1768
...,...,...
1878,3345.544299,200560
1879,3345.711110,200570
1880,3372.316661,202165
1881,3562.708709,213579


In [16]:
behavior_session.stimulus_presentations

,start_time,stop_time,duration,image_name,image_index,is_change,omitted,start_frame,end_frame,image_set
stimulus_presentations_id,,,,,,,,,,
0,20.037870,20.288087,0.250217,im065,0,False,False,1200.0,1215.0,Natural_Images_Lum_Matched_set_training_2017.0...
1,20.788495,21.038715,0.250220,im065,0,False,False,1245.0,1260.0,Natural_Images_Lum_Matched_set_training_2017.0...
2,21.539117,21.789369,0.250252,im065,0,False,False,1290.0,1305.0,Natural_Images_Lum_Matched_set_training_2017.0...
3,22.289746,22.539996,0.250250,im065,0,False,False,1335.0,1350.0,Natural_Images_Lum_Matched_set_training_2017.0...
4,23.040379,23.290586,0.250207,im065,0,False,False,1380.0,1395.0,Natural_Images_Lum_Matched_set_training_2017.0...
...,...,...,...,...,...,...,...,...,...,...
4791,3617.204142,3617.454360,0.250218,im063,4,False,False,216846.0,216861.0,Natural_Images_Lum_Matched_set_training_2017.0...
4792,3617.954771,3618.204983,0.250212,im063,4,False,False,216891.0,216906.0,Natural_Images_Lum_Matched_set_training_2017.0...
4793,3618.705398,3618.955610,0.250211,im063,4,False,False,216936.0,216951.0,Natural_Images_Lum_Matched_set_training_2017.0...


In [14]:
behavior_session.trials

,initial_image_name,change_image_name,stimulus_change,change_time,go,catch,lick_times,response_time,response_latency,reward_time,...,hit,false_alarm,miss,correct_reject,aborted,auto_rewarded,change_frame,start_time,stop_time,trial_length
trials_id,,,,,,,,,,,,,,,,,,,,,
0,im065,im065,False,NaN,False,False,[20.6216912999948],NaN,NaN,NaN,...,False,False,False,False,True,False,NaN,20.021189,20.938623,0.917434
1,im065,im065,False,NaN,False,False,[23.640902700004027],NaN,NaN,NaN,...,False,False,False,False,True,False,NaN,21.522439,23.957808,2.435369
2,im065,im061,True,29.062116,False,False,"[29.512453099988726, 29.679261500000813]",29.512453,0.450337,29.212202,...,False,False,False,False,False,True,1741.0,24.524951,33.332310,8.807359
3,im061,im066,True,37.318970,False,False,"[41.33899829997923, 41.48912939998214, 41.6058...",41.338998,4.020029,37.452413,...,False,False,False,False,False,True,2236.0,33.549161,41.572536,8.023375
4,im066,im066,False,NaN,False,False,"[41.90614399999273, 42.022902199983456, 42.223...",NaN,NaN,NaN,...,False,False,False,False,True,False,NaN,41.806059,43.774370,1.968311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,im066,im069,True,3580.406801,True,False,[],NaN,inf,NaN,...,False,False,True,False,False,False,214640.0,3575.886361,3584.677030,8.790668
609,im069,im066,True,3591.666190,True,False,[],NaN,inf,NaN,...,False,False,True,False,False,False,215315.0,3584.893877,3595.919778,11.025901
610,im066,im085,True,3601.424323,True,False,[],NaN,inf,NaN,...,False,False,True,False,False,False,215900.0,3596.153263,3605.677870,9.524606


In [17]:
behavior_session.stimulus_templates

,unwarped,warped
image_name,,
im065,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[198, 197, 201, 207, 207, 202, 198, 201, 207,..."
im077,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[129, 135, 140, 142, 140, 135, 129, 124, 117,..."
im066,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[83, 85, 82, 78, 78, 81, 84, 84, 82, 82, 84, ..."
im061,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[125, 126, 126, 126, 125, 125, 125, 125, 125,..."
im063,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[105, 108, 107, 101, 97, 96, 98, 101, 103, 10..."
im062,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[103, 103, 103, 105, 112, 125, 145, 172, 204,..."
im085,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[54, 56, 50, 45, 45, 51, 56, 55, 51, 48, 49, ..."
im069,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[251, 248, 255, 255, 255, 254, 246, 248, 255,..."


In [12]:
behavior_session.eye_tracking

AttributeError: 'BehaviorSession' object has no attribute 'eye_tracking'

womp womp, no eye tracking for behavior sessions. unfortunately it is only accessible via the ophys_experiments, which are problematic for this mouse, but here is some code to attempt to load one anyway. note that some experiments will fail to load due to other issues with ophys data processing

### get ophys_experiment_table 

`ophys_experiment_table` contains a record of each unique 2P field of view recorded in each session. 
One `ophys_session_id` can have 8 imaging planes, corresonding to 8 `ophys_experiment_ids`. 
A given imaging plane that is recorded across multiple sessions is linked by its `ophys_container_id`. 

In [29]:
experiments_table = cache.get_ophys_experiment_table(passed_only=False)

experiments = experiments_table[experiments_table.project_code=='LearningmFISHTask1A']
experiments.head()

,equipment_name,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,cre_line,...,session_name,isi_experiment_id,imaging_depth,targeted_structure,published_at,date_of_acquisition,session_type,experience_level,passive,image_set
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
1154288458,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,165,VISp,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288461,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,265,VISp,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288463,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,170,VISl,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288465,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,270,VISl,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i
1154288467,MESO.2,1142290477,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,603892,Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg,"[Slc32a1-T2A-FlpO, Gad2-IRES-Cre]",M,116.0,130a7f92-a251-4aa0-abdb-90683d6be53e,Gad2-IRES-Cre,...,20220126_603892_training2,1145918593,175,VISal,NaT,2022-01-26 10:42:34,TRAINING_2_gratings_flashed,None,False,i


### get ophys containers for special mouse 

In [30]:
mouse_data = experiments[experiments.mouse_id=='603892'].sort_values(by='date_of_acquisition')
ophys_container_ids = mouse_data.ophys_container_id.unique()
len(ophys_container_ids)

8

### pick a container and experiment 

In [28]:
ophys_container_id = 1157145092
experiment_id = experiments[experiments.ophys_container_id==ophys_container_id].index.values[9]
experiment_id

1155949168

### get ophys dataset for this experiment and look at eye tracking

In [31]:
ophys_experiment = cache.get_behavior_ophys_experiment(experiment_id)

C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\behavior\data_objects\running_speed\running_processing.py:368: UserWarning:

Time array is 1 value shorter than encoder array. Last encoder value removed


C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\behavior\data_objects\metadata\subject_metadata\reporter_line.py:111: UserWarning:

Could not parse indicator from reporter because noneof the expected substrings were found in the reporter



(33,)


C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\behavior\data_objects\eye_tracking\eye_tracking_table.py:233: UserWarning:


in sync_file: \\allen\programs\mindscope\production\learning\prod0\specimen_1142290487\ophys_session_1155848487\1155848487_20220203T114150.h5
Error! The number of sync file frame times (273014) does not match the number of eye tracking frames (273015)!
returning empty eye_tracking DataFrame

C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\behavior\data_objects\metadata\subject_metadata\reporter_line.py:111: UserWarning:

Could not parse indicator from reporter because noneof the expected substrings were found in the reporter



(33,)


C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\behavior\data_objects\eye_tracking\eye_tracking_table.py:233: UserWarning:


in sync_file: \\allen\programs\mindscope\production\learning\prod0\specimen_1142290487\ophys_session_1155848487\1155848487_20220203T114150.h5
Error! The number of sync file frame times (273014) does not match the number of eye tracking frames (273015)!
returning empty eye_tracking DataFrame



In [32]:
ophys_experiment.eye_tracking

,timestamps,cr_area,eye_area,pupil_area,likely_blink,pupil_area_raw,cr_area_raw,eye_area_raw,cr_center_x,cr_center_y,...,eye_center_x,eye_center_y,eye_width,eye_height,eye_phi,pupil_center_x,pupil_center_y,pupil_width,pupil_height,pupil_phi
frame,,,,,,,,,,,,,,,,,,,,,


Whoops well i guess we cant look at eye tracking here either!! The above sync file timing error is being tracked here, but has not yet been resolved:
    
    https://github.com/AllenInstitute/AllenSDK/issues/2307